In [1]:
import numpy as np
import pandas as pd
import pickle
import utils
import time
import tensorflow as tf

/home/davidhou8791/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
emoji_vectors = pickle.load(open('data/emoji_vectors.p', 'rb'))

In [3]:
w2v_vectors = pickle.load(open('data/w2v_vectors.p', 'rb'))

In [4]:
list(emoji_vectors.keys())[0:10]

['🙅', '💂', '🏃', '🤸', '🏌', '0', '👨', '👏', '🚶', '🦑']

In [5]:
list(w2v_vectors.keys())[0:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

In [6]:
emoji_embedding = np.array([v for v in emoji_vectors.values()])

In [7]:
emoji_embedding.shape

(1126, 300)

In [8]:
utils.cosine_similarity(emoji_vectors['⭐'], w2v_vectors['star'])

1.0

In [9]:
sess = tf.InteractiveSession()

In [10]:
def matmul3d(X, W):
    """Wrapper for tf.matmul to handle a 3D input tensor X.
    Will perform multiplication along the last dimension.

    Args:
      X: [m,n,k]
      W: [k,l]

    Returns:
      XW: [m,n,l]
    """
    Xr = tf.reshape(X, [-1, tf.shape(X)[2]])
    XWr = tf.matmul(Xr, W)
    newshape = [tf.shape(X)[0], tf.shape(X)[1], tf.shape(W)[1]]
    return tf.reshape(XWr, newshape)

In [66]:
# tf.reset_default_graph()
# sess.close()
sess = tf.InteractiveSession()

In [67]:
batch_size = 128
nodes = 300
embed_size = 300
x_seq_length = 10

inputs = tf.placeholder(tf.float32, (None, x_seq_length, embed_size), 'inputs')
input_mean = tf.nn.l2_normalize(tf.reduce_mean(inputs, axis=1), axis=1, name='input_mean')

output_embedding = tf.constant(emoji_embedding, name='output_embedding')

with tf.name_scope('network'):
    lstm_cell = tf.contrib.rnn.LSTMCell(nodes, name='lstm')
    lstm_outputs, _ = tf.nn.dynamic_rnn(lstm_cell, inputs=inputs, dtype=tf.float32)

    logits = tf.layers.dense(lstm_outputs, units=len(emoji_vectors), activation='softmax', name='dense') 
    outputs = matmul3d(logits, output_embedding)

    output_mean = tf.nn.l2_normalize(tf.reduce_mean(outputs, axis=1), axis=1)

with tf.name_scope("optimization"):
    loss = tf.losses.cosine_distance(input_mean, output_mean, axis=1)
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('models/lstm_toy_example', sess.graph)

In [12]:
X = np.array([[w2v_vectors['star']]*x_seq_length for _ in range(batch_size)])
X.shape

(128, 10, 300)

In [13]:
lstm_outputs[-1][-1].shape

TensorShape([Dimension(300)])

In [68]:
sess.run(tf.global_variables_initializer())
epochs = 50
for i in range(epochs):
    start_time = time.time()
    _, l, summary = sess.run([optimizer, loss, merged], feed_dict={inputs:X})
    writer.add_summary(summary, global_step=i)
    if l < .0005:
        print('Epoch {:3} Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, l, time.time() - start_time))
        break
    elif not i%10:
        print('Epoch {:3} Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, l, time.time() - start_time))
    

Epoch   0 Loss:  0.757 Epoch duration:  0.722s
Epoch  10 Loss:  0.213 Epoch duration:  0.636s
Epoch  20 Loss:  0.001 Epoch duration:  0.632s
Epoch  30 Loss:  0.001 Epoch duration:  0.633s
Epoch  33 Loss:  0.000 Epoch duration:  0.636s


In [16]:
lo = sess.run(logits, feed_dict={inputs: np.array([w2v_vectors['star']]*x_seq_length).reshape(-1, 10, 300)})

In [17]:
lo.shape

(1, 10, 1126)

In [18]:
np.argmax(lo, axis=2)

array([[984, 984, 984, 984, 984, 984, 984, 984, 984, 984]])

In [19]:
list(emoji_vectors.keys())[984]

'⭐'

In [25]:
saver = tf.train.Saver()

saver.save(sess, 'models/lstm_toy_example')